In [1]:
import numpy as np
from scipy.integrate import odeint
import random
from keras import *
from keras.layers import *
import matplotlib.pyplot as plt


Using TensorFlow backend.


In [2]:
#NUM_SEQ_POINTS are RANGE Parameters used for generating test and train data
NUM_SEQ_POINTS = 50
RANGE = 6
NUM_TEST_STARTING_POINTS = 5
#RHO, SIGMA, and BETA are parameters for the Lorenz system
RHO = 28.0
SIGMA = 10.0
BETA = 8.0 / 3.0

#NUM_EPOCS are BATCH_SIZE is the parameters used for training the neural network
NUM_EPOCS = 50
BATCH_SIZE = 100

In [3]:
# the derivitives dx/dt , dy/dt, and dz/dt in Lorenz system are defined in function f, 
# to be further used for solving the system, and getting the sample points
def f(state, t):
    global RHO
    global SIGMA
    global BETA
    x, y, z = state  # Unpack the state vector
    return SIGMA * (y - x), x * (RHO - z) - y, x * y - BETA * z  # Derivatives


In [4]:
def generate_train_data():
    global NUM_POINTS
    global RANGE
    init_states=[]
    #domain range consists of points from [RANGE,RANGE)
    domain_range=[(k-(int(RANGE/2))) for k in range(RANGE)]
    
    #initial states are generated for Lorenz system by getting points with x,y,z 
    #each one in the domain range
    for i in domain_range:
        for j in domain_range:
            for k in domain_range:
                init_states.append([i,j,k])
                init_states.append([-i,-j,-k])
    init_states=np.array(init_states)
    print(init_states.shape)
    #each coordinate can get RANGE number of values, and since we include negative values for each one, 
    #total number of initial states equal to 2*RANGE*RANGE*RANGE
    num_starting_points = 2* RANGE*RANGE*RANGE
    
    #the system is solved with t=0 to t=40 with a small time step of dt=0.01
    t = np.arange(0.0, 40.0, 0.01)
    #for each initial point, we solve the sytem and take NUM_SEQ_POINTS points and add them to the data set
    X_train= np.zeros((num_starting_points*NUM_SEQ_POINTS,3))
    Y_train= np.zeros((num_starting_points*NUM_SEQ_POINTS,3))
    cnt=1
    for state0 in init_states: 
        print(cnt)
        #odeint function is used to solve the Lorenz system with specified parameters, and the initial state, in
        #the given time period 
        states = odeint(f, state0, t)
        X_train[(cnt-1)*NUM_SEQ_POINTS:cnt*NUM_SEQ_POINTS,:] = \
            states[:NUM_SEQ_POINTS,:]
        #the value of Y for each point xt is the its next state xt+1 which is the next point in the sequence 
        Y_train[(cnt-1)*NUM_SEQ_POINTS:cnt*NUM_SEQ_POINTS,:] = \
            states[1:(NUM_SEQ_POINTS+1),:]
        cnt+=1
    return X_train, Y_train


In [5]:
def generate_test_data():
    global RANGE
    global NUM_TEST_STARTING_POINTS
    init_states=[]
    #for generating the test data, initial states get generated in the range of (-RANGE,RANGE)
    #using a uniform random distribution
    for i in range(NUM_TEST_STARTING_POINTS):
        x= random.uniform(-RANGE, RANGE) 
        y= random.uniform(-RANGE, RANGE) 
        z= random.uniform(-RANGE, RANGE) 
        init_states.append([x,y,z])
    init_states=np.array(init_states)
    #state0 = [1.0, 1.0, 1.0]
    t = np.arange(0.0, 40.0, 0.01)
    X_test= np.zeros((NUM_TEST_STARTING_POINTS*NUM_SEQ_POINTS,3))
    Y_test= np.zeros((NUM_TEST_STARTING_POINTS*NUM_SEQ_POINTS,3))
    cnt=1
    for state0 in init_states: 
        states = odeint(f, state0, t)
        X_test[(cnt-1)*NUM_SEQ_POINTS:cnt*NUM_SEQ_POINTS,:] = \
            states[:NUM_SEQ_POINTS,:]
        Y_test[(cnt-1)*NUM_SEQ_POINTS:cnt*NUM_SEQ_POINTS,:] = \
            states[1:(NUM_SEQ_POINTS+1),:]
        cnt+=1
    return X_test, Y_test

In [6]:
X_train, Y_train = generate_train_data()
X_test, Y_test = generate_test_data()

(432, 3)
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275

In [7]:
print(Y_test.shape)

(250, 3)


In [8]:
#model = Sequential()
dt= 0.01

#the function that outputs a neural network which is trained using the Euler's method

def get_model_euler():
    global dt
    input_layer = Input(shape=(3,))
    hidden_1= Dense(16, activation='relu')(input_layer)
    hidden_2= Dense(64, activation='relu')(hidden_1)
    hidden_3= Dense(64, activation='relu')(hidden_2)
    #the deriv layer outputs the derivative of the input, then its output is multiplied by dt in an internal layer
    #and then in another layer x0 is added to the dt*f'(x), so the prediction of the network would be x0 + dt*f'(x0)
    deriv_layer= Dense(3, activation=None)(hidden_3)
    out = Lambda(lambda x: x * dt)(deriv_layer)
    out = layers.Add()([out,input_layer])
    model = Model(input_layer, out)
    return model




In [9]:
model = get_model_euler()
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])


In [10]:
history=model.fit(X_train, Y_train, epochs=NUM_EPOCS, batch_size=BATCH_SIZE, validation_data=(X_test, Y_test))


Train on 21600 samples, validate on 250 samples
Epoch 1/50
21600/21600 [==============================] - 2s 115us/step - loss: 1.0003 - mse: 1.0003 - val_loss: 0.2850 - val_mse: 0.2850
Epoch 2/50
21600/21600 [==============================] - 1s 40us/step - loss: 0.0404 - mse: 0.0404 - val_loss: 0.1485 - val_mse: 0.1485
Epoch 3/50
21600/21600 [==============================] - 1s 39us/step - loss: 0.0184 - mse: 0.0184 - val_loss: 0.1390 - val_mse: 0.1390
Epoch 4/50
21600/21600 [==============================] - 1s 43us/step - loss: 0.0153 - mse: 0.0153 - val_loss: 0.1286 - val_mse: 0.1286
Epoch 5/50
21600/21600 [==============================] - 1s 41us/step - loss: 0.0128 - mse: 0.0128 - val_loss: 0.1170 - val_mse: 0.1170
Epoch 6/50
21600/21600 [==============================] - 1s 41us/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1019 - val_mse: 0.1019
Epoch 7/50
21600/21600 [==============================] - 1s 40us/step - loss: 0.0091 - mse: 0.0091 - val_loss: 0.0906 - val_mse:

In [8]:
#function that applies the derrivites learned by neural network to get the predictions
#initial_state is a 3d point, model is the the trained model, and num_of_predicted is number of next points
#to be predicted
def get_prediction(initial_state,model,num_of_predicted):
    predicted_points=[]

    p = initial_state
    p = np.array(p)
    predicted_points.append(p[0])
    last_point=p
    for i in range(num_of_predicted):
        next_point = model.predict(last_point)
        predicted_points.append(next_point[0])
        last_point= next_point
    print(predicted_points)
    np.savetxt("predicted.txt",predicted_points)
    return predicted_points


In [16]:
points = get_prediction([[10,10,10]],model,2000)


[ 9.850765 11.193816 10.717509]
[ 9.859667 12.377114 11.54639 ]
[10.013049 13.486114 12.498118]
[10.278336 14.49682  13.595644]
[10.628417  15.3827505 14.847591 ]
[11.040441 16.108236 16.269379]
[11.49197  16.63405  17.845808]
[11.950856 16.917074 19.570396]
[12.394372 16.901299 21.448261]
[12.796004 16.553812 23.384949]
[13.11529  15.865559 25.241564]
[13.326741 14.822913 26.95071 ]
[13.404565 13.41411  28.454395]
[13.330521 11.688465 29.67616 ]
[13.081895  9.687119 30.57307 ]
[12.665923  7.484791 31.143433]
[12.070529  5.197362 31.318798]
[11.292491   3.0934203 31.076454 ]
[10.370696  1.280275 30.44821 ]
[ 9.36879    -0.20507836 29.51069   ]
[ 8.3345785 -1.407486  28.431034 ]
[ 7.2690654 -2.3836308 27.296762 ]
[ 6.2264624 -3.119995  26.136993 ]
[ 5.228004 -3.641544 24.97272 ]
[ 4.268829  -4.0346155 23.848085 ]
[ 3.3627818 -4.3522267 22.774725 ]
[ 2.5170321 -4.593944  21.74682  ]
[ 1.7369817 -4.7480564 20.765884 ]
[ 1.0240271 -4.844159  19.83408  ]
[ 0.3761372 -4.893158  18.949575 ]
[

[ 4.594532   4.7919135 20.425688 ]
[ 4.6679397  4.545327  19.957405 ]
[ 4.7006154  4.300934  19.4675   ]
[ 4.697548   4.0589614 18.95355  ]
[ 4.6631517  3.8196592 18.412918 ]
[ 4.6028366  3.584462  17.842072 ]
[ 4.5195403  3.3542504 17.23559  ]
[ 4.422007  3.137097 16.59748 ]
[ 4.317813   2.9325957 15.931682 ]
[ 4.209839  2.743006 15.240688]
[ 4.1003337  2.5846448 14.5316725]
[ 3.982927   2.4849403 13.823439 ]
[ 3.8632329  2.4689689 13.118314 ]
[ 3.752815   2.5334044 12.420105 ]
[ 3.6636877  2.6787696 11.740794 ]
[ 3.6055505  2.9072955 11.107785 ]
[ 3.5724308  3.21232   10.535681 ]
[ 3.5720594  3.5953689 10.019007 ]
[3.61275  4.059289 9.55704 ]
[3.6930559 4.602625  9.159322 ]
[3.8154762 5.2257557 8.833529 ]
[3.9825075 5.9338994 8.573252 ]
[4.204435  6.700569  8.3763075]
[4.48311   7.5333223 8.248309 ]
[4.826026 8.423798 8.196392]
[5.2304435 9.364277  8.240924 ]
[ 5.6876106 10.359813   8.3866825]
[ 6.1978645 11.430889   8.652818 ]
[ 6.764015  12.579851   9.0498495]
[ 7.400112 13.779976 

[-12.461139 -20.61439   18.74654 ]
[-13.28139  -20.954185  21.242464]
[-14.032341 -20.867594  23.893324]
[-14.674276 -20.296484  26.624184]
[-15.183703 -19.211773  29.332031]
[-15.52293  -17.595404  31.865623]
[-15.649676 -15.438297  34.12477 ]
[-15.481457 -12.789546  35.834156]
[-15.04356   -9.881808  36.895603]
[-14.367119   -6.9332333  37.294205 ]
[-13.500736  -4.217637  37.070816]
[-12.4568405  -1.8223269  36.365997 ]
[-11.264798     0.21405697  35.259434  ]
[-10.015547   1.889181  33.9212  ]
[-8.765745   3.1798058 32.470634 ]
[-7.5534716  4.1271205 30.992683 ]
[-6.3971725  4.761525  29.538504 ]
[-5.289507  5.173139 28.155466]
[-4.2351656  5.4189444 26.863518 ]
[-3.2483366  5.526014  25.655369 ]
[-2.352353  5.538249 24.510124]
[-1.5423563  5.469063  23.428522 ]
[-0.81357294  5.3323693  22.418232  ]
[-0.16412973  5.136633   21.475645  ]
[ 0.4083894  4.8963776 20.596275 ]
[ 0.9118286  4.6318    19.764942 ]
[ 1.3503023  4.3448052 18.975866 ]
[ 1.7191033  4.0502787 18.219913 ]
[ 2.0214

[-9.20695   -6.6360807 21.017096 ]
[-8.801591  -5.5122557 20.922306 ]
[-8.309632  -4.4730883 20.589933 ]
[-7.7673244 -3.6159678 20.045336 ]
[-7.2077084 -2.9597788 19.364914 ]
[-6.639366 -2.458777 18.612452]
[-6.081257 -2.118342 17.817238]
[-5.5492554 -1.935575  16.99698  ]
[-5.055881  -1.8878363 16.168213 ]
[-4.608367  -1.9542242 15.350669 ]
[-4.2085423 -2.1075532 14.558707 ]
[-3.8680692 -2.3084214 13.800845 ]
[-3.5825784 -2.5379398 13.074173 ]
[-3.346655  -2.7929764 12.380219 ]
[-3.1703959 -3.0944486 11.71639  ]
[-3.058108  -3.4498918 11.093854 ]
[-3.0051293 -3.8534505 10.523126 ]
[-3.0087729 -4.2981505 10.008021 ]
[-3.0765603 -4.783268   9.557165 ]
[-3.2043972 -5.305553   9.174332 ]
[-3.4007742 -5.8560824  8.844698 ]
[-3.637145 -6.458218  8.551746]
[-3.9143798 -7.099611   8.305686 ]
[-4.233521  -7.7990336  8.125517 ]
[-4.6004295 -8.564592   8.014666 ]
[-5.0122867 -9.395942   7.9919915]
[ -5.4744596 -10.301577    8.076472 ]
[ -5.9922233 -11.283025    8.282497 ]
[ -6.569134 -12.333904 

[ -9.674049 -10.73597   18.823963]
[ -9.81408  -10.194375  19.703087]
[-9.838289 -9.434236 20.441898]
[-9.735302 -8.461503 20.992775]
[-9.513117  -7.3477316 21.30224  ]
[-9.163347  -6.1575985 21.356852 ]
[-8.702033  -5.0069866 21.15575  ]
[-8.173988 -4.008417 20.714476]
[-7.609788 -3.219416 20.086687]
[-7.027525  -2.6049669 19.350914 ]
[-6.444304  -2.1530967 18.557602 ]
[-5.8799477 -1.8675475 17.73011  ]
[-5.3471622 -1.7346699 16.885    ]
[-4.8576126 -1.7266507 16.04166  ]
[-4.416462  -1.8259169 15.213702 ]
[-4.03047   -1.9894556 14.4178095]
[-3.7010665 -2.1919193 13.654864 ]
[-3.4250777 -2.4215424 12.923599 ]
[-3.1951675 -2.6746027 12.233016 ]
[-3.0229275 -2.9698505 11.573572 ]
[-2.9134398 -3.3150291 10.953964 ]
[-2.8625352 -3.7141464 10.3842125]
[-2.867064  -4.1544237  9.869565 ]
[-2.9312387 -4.6309276  9.416501 ]
[-3.0554752 -5.142463   9.031209 ]
[-3.2440202 -5.678633   8.700392 ]
[-3.4735172 -6.2648087  8.40633  ]
[-3.7420628 -6.8958025  8.154828 ]
[-4.051495  -7.5787134  7.962645

[ 6.8473043 -5.726453  32.59596  ]
[ 5.5661926 -6.3448005 31.196545 ]
[ 4.3550324 -6.7447543 29.889862 ]
[ 3.2309535 -6.989865  28.686293 ]
[ 2.1986158 -7.1181135 27.58664  ]
[ 1.2667565 -7.170407  26.584217 ]
[ 0.4310097 -7.156142  25.664467 ]
[-0.31484216 -7.082409   24.819859  ]
[-0.9893863 -7.007753  24.056482 ]
[-1.6005898 -6.9389124 23.368423 ]
[-2.1551752 -6.8737316 22.744331 ]
[-2.6576052 -6.8074684 22.176723 ]
[-3.0998833 -6.7229085 21.65024  ]
[-3.4860985 -6.619684  21.159077 ]
[-3.819887 -6.49736  20.697765]
[-4.101022  -6.3690705 20.237652 ]
[-4.327139 -6.268006 19.757011]
[-4.5066    -6.2057776 19.247921 ]
[-4.6493034 -6.1755705 18.717455 ]
[-4.7626634 -6.175711  18.169027 ]
[-4.856464  -6.2100687 17.611128 ]
[-4.9398656 -6.283337  17.05001  ]
[-5.0196004 -6.3984346 16.489574 ]
[-5.1062784 -6.5706    15.944592 ]
[-5.210535 -6.802483 15.430462]
[-5.338971  -7.0810256 14.959982 ]
[-5.490999 -7.379857 14.548223]
[-5.664629  -7.6879883 14.201662 ]
[-5.864803 -7.999463 13.92325

[ -7.305057 -13.560418   9.289414]
[ -7.9949923 -14.739025   10.024265 ]
[ -8.733347  -15.9710865  10.985756 ]
[ -9.518847 -17.20179   12.213311]
[-10.33716  -18.350952  13.73965 ]
[-11.177106 -19.347866  15.56783 ]
[-12.018444 -20.13388   17.683825]
[-12.842221 -20.619064  20.04804 ]
[-13.617219 -20.716162  22.60102 ]
[-14.298637 -20.361515  25.262255]
[-14.858189 -19.507198  27.953255]
[-15.267579 -18.137169  30.5265  ]
[-15.484662 -16.238934  32.87851 ]
[-15.458825 -13.826419  34.840317]
[-15.12035  -11.023947  36.164352]
[-14.546107   -8.1155615  36.822254 ]
[-13.769031   -5.2880006  36.82871  ]
[-12.80269    -2.7862067  36.319046 ]
[-11.680803   -0.6271422  35.38412  ]
[-10.460144   1.179668  34.133495]
[-9.218771  2.615178 32.733368]
[-8.001289   3.6816754 31.26377  ]
[-6.8327723  4.434767  29.807827 ]
[-5.717354  4.92849  28.401926]
[-4.653003   5.2340403 27.077913 ]
[-3.6459785  5.376012  25.845394 ]
[-2.7223608  5.420708  24.679302 ]
[-1.8876257  5.3779078 23.57211  ]
[-1.1349

In [19]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
def plot_trajectories(points):
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.plot(points[:, 0], points[:, 1], points[:, 2])
    plt.savefig("euler_trained.png")
    plt.show()

In [ ]:
plot_trajectories(points)

In [ ]:
 
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.yscale('log')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper right')
plt.show()

In [7]:

dt= 0.01

def get_model_rk4():
    global dt
    #first the derivative layer is defined as a module without wiring, then for coumputation 
    #of k1 through k4 appropriate inputs are passed to the module as input 
    input_1 = Input(shape=(3,))
    hidden_1_g= Dense(16, activation='relu')
    hidden_2_g= Dense(64, activation='relu')
    hidden_3_g= Dense(64, activation='relu')
    out_g= Dense(3, activation=None)
    
    #g1_out equals to the value of g(x0) which is the derivitive of x0
    g1_out= hidden_1_g(input_1)
    g1_out= hidden_2_g(g1_out)
    g1_out= hidden_3_g(g1_out)
    g1_out=out_g(g1_out)
    #lambda is used to multiply the result of the derivitive by dt
    k1 = Lambda(lambda x: x * dt)(g1_out)

    k1_div_2= Lambda(lambda x: x/2)(k1)
    input_2 = layers.Add()([input_1,k1_div_2])
    g2_out=hidden_1_g(input_2)
    g2_out=hidden_2_g(g2_out)
    g2_out=hidden_3_g(g2_out)
    g2_out=out_g(g2_out)

    k2 = Lambda(lambda x: x * dt)(g2_out)

    k2_div_2 = Lambda(lambda x: x/2)(k2)
    input_3 = layers.Add()([input_1,k2_div_2])
    g3_out=hidden_1_g(input_3)
    g3_out=hidden_2_g(g3_out)
    g3_out=hidden_3_g(g3_out)
    g3_out=out_g(g3_out)

    k3= Lambda(lambda x: x * dt)(g3_out)

    input_4=layers.Add()([input_1,k3])
    g4_out=hidden_1_g(input_4)
    g4_out=hidden_2_g(g4_out)
    g4_out=hidden_3_g(g4_out)
    g4_out=out_g(g4_out)

    k4= Lambda(lambda x: x * dt)(g3_out)

    out = layers.Add()([input_1, Lambda(lambda x: x/6)(k1),Lambda(lambda x: x/3)(k2),Lambda(lambda x: x/3)(k3),Lambda(lambda x: x/6)(k4)])
    model = Model(input_1, out)
    return model
model = get_model_rk4()
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])
history=model.fit(X_train, Y_train, epochs=NUM_EPOCS, batch_size=BATCH_SIZE, validation_data=(X_test, Y_test))


#added = layers.Add()([o_mult_dt,i])

Train on 21600 samples, validate on 250 samples
Epoch 1/50
21600/21600 [==============================] - 4s 189us/step - loss: 0.9593 - mse: 0.9593 - val_loss: 0.1629 - val_mse: 0.1629
Epoch 2/50
21600/21600 [==============================] - 2s 78us/step - loss: 0.0399 - mse: 0.0399 - val_loss: 0.0800 - val_mse: 0.0800
Epoch 3/50
21600/21600 [==============================] - 2s 107us/step - loss: 0.0178 - mse: 0.0178 - val_loss: 0.0641 - val_mse: 0.0641
Epoch 4/50
21600/21600 [==============================] - 2s 102us/step - loss: 0.0137 - mse: 0.0137 - val_loss: 0.0513 - val_mse: 0.0513
Epoch 5/50
21600/21600 [==============================] - 2s 113us/step - loss: 0.0112 - mse: 0.0112 - val_loss: 0.0439 - val_mse: 0.0439
Epoch 6/50
21600/21600 [==============================] - 2s 113us/step - loss: 0.0095 - mse: 0.0095 - val_loss: 0.0396 - val_mse: 0.0396
Epoch 7/50
21600/21600 [==============================] - 2s 109us/step - loss: 0.0084 - mse: 0.0084 - val_loss: 0.0339 - val

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.yscale('log')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper right')
plt.show()

In [9]:
states = get_prediction([[2.5,2.5,2.5]],model,4000)

[array([10, 10, 10]), array([10.161911, 10.841219, 10.843656], dtype=float32), array([10.382056, 11.623008, 11.76655 ], dtype=float32), array([10.65529 , 12.315536, 12.79705 ], dtype=float32), array([10.960356, 12.892   , 13.968685], dtype=float32), array([11.287865, 13.309131, 15.310923], dtype=float32), array([11.627672, 13.502033, 16.849745], dtype=float32), array([11.966801, 13.413846, 18.610134], dtype=float32), array([12.252469, 13.016332, 20.413847], dtype=float32), array([12.455115, 12.269903, 22.193073], dtype=float32), array([12.525314, 11.131567, 23.859476], dtype=float32), array([12.408062,  9.59296 , 25.225435], dtype=float32), array([12.0980215,  7.753973 , 26.207573 ], dtype=float32), array([11.627858,  5.817865, 26.753853], dtype=float32), array([11.008252 ,  3.8488965, 26.854599 ], dtype=float32), array([10.260979 ,  2.0225036, 26.527617 ], dtype=float32), array([ 9.405499  ,  0.44186437, 25.868053  ], dtype=float32), array([ 8.480054  , -0.88991725, 24.996775  ], dtyp

In [10]:
plot_trajectories(states)